In [1]:
import argparse
import numpy as np
import matplotlib.pyplot as plt
from mayavi import mlab 

In [2]:
def generate_cot_matrix(vertices, triangles):
    
    numv = vertices.shape[0]
    numt = triangles.shape[0]

    cot_matrix = np.zeros((numv, numv))
    scaling_factor = np.ones((numv, numv))

    for i in range(numv):
        
        req_t = triangles[(triangles[:, 0] == i) | (triangles[:, 1] == i) | (triangles[:, 2] == i)]
        
        for j in range(req_t.shape[0]):
            
                
            nbhr = [v for v in req_t[j] if v != i]

            vec1 = (vertices[nbhr[0]] - vertices[i]) / np.linalg.norm(vertices[nbhr[0]] - vertices[i], 2)
            vec2 = (vertices[nbhr[1]] - vertices[i]) / np.linalg.norm(vertices[nbhr[1]] - vertices[i], 2)
            angle_at_x = np.arccos(np.dot(vec1, vec2))

            if angle_at_x > np.pi / 2:
                scaling_factor[i, nbhr[0]] = 1 / 2



            vec1a = (vertices[i] - vertices[nbhr[0]]) / np.linalg.norm(vertices[i] - vertices[nbhr[0]], 2)
            vec2a = (vertices[nbhr[1]] - vertices[nbhr[0]]) / np.linalg.norm(vertices[nbhr[1]] - vertices[nbhr[0]], 2)

            inner_prod = np.dot(vec1a, vec2a)
            angle = np.arccos(inner_prod)



            if angle > np.pi / 2:
                scaling_factor[i, nbhr[0]] = 1 / 4

            cot_matrix[i, nbhr[1]] += 1 / np.tan(angle)

            vec1b = (vertices[i] - vertices[nbhr[1]]) / np.linalg.norm(vertices[i] - vertices[nbhr[1]], 2)
            vec2b = (vertices[nbhr[0]] - vertices[nbhr[1]]) / np.linalg.norm(vertices[nbhr[0]] - vertices[nbhr[1]], 2)

            inner_prod = np.dot(vec1b, vec2b)
            angle = np.arccos(inner_prod)

            if angle > np.pi / 2:
                scaling_factor[i, nbhr[0]] = 1 / 4

            cot_matrix[i, nbhr[0]] += 1 / np.tan(angle)
    
    return cot_matrix, scaling_factor
 

In [3]:
def get_diff_norm(vertices):
    numv = vertices.shape[0]
    numt = triangles.shape[0]
    diff_norm = np.zeros((numv, numv))
    diff_norm = np.linalg.norm(vertices.reshape(-1, 1, 3) - vertices, 2, axis=2)**2
#     for i in range(numv):
#         for j in range(numv):
#             diff_norm[i, j] = np.linalg.norm(vertices.reshape(-1, 1, 3) - vertices, 2, axis=2)**2  
    return diff_norm

In [4]:
def calc_A_mixed(vertices, cot_matrix, scaling_factor):
#     cot_matrix, scaling_factor = generate_cot_matrix(vertices, triangles)
    diff_norm = get_diff_norm(vertices)
    A_mixed = np.sum(cot_matrix * diff_norm * scaling_factor, axis = 1) / 8
    return A_mixed

In [5]:
def get_diff(vertices):
    numv = vertices.shape[0]
    numt = triangles.shape[0]
    diff_x = np.zeros((numv, numv, 3))
    diff_x = vertices.reshape(-1, 1, 3) - vertices
#     for i in range(numv):
#         for j in range(numv):
#             diff_x[i, j] = vertices[i] - vertices[j]
    return diff_x

In [6]:
def mean_curvature_normal_operator(vertices, triangles, A_mixed, cot_matrix):
    numv = vertices.shape[0]
    numt = triangles.shape[0]
    diff_x = get_diff(vertices)
    K = np.zeros((numv, numv, 3))
#     K = np.multiply(diff_x, cot_matrix)
    for i in range(numv):
#         for j in range(numv):
        K[i, :] = (cot_matrix[i, :] * diff_x[i, :].T).T / (2 * A_mixed[i])
    K = np.sum(K, axis=1)
    return K

In [7]:
def get_mean_curvature(vertices, triangles, A_mixed, cot_matrix):
    K = mean_curvature_normal_operator(vertices, triangles, A_mixed, cot_matrix)
    K_H = 0.5 * np.linalg.norm(K, 2, axis=1)
    return K_H

In [8]:
def get_gaussian_curvature(vertices, triangles, A_mixed):
    numv = vertices.shape[0]
    numt = triangles.shape[0]
    K_G = np.zeros(numv)
    for i in range(numv):
        sum_theta = 0
        req_t = triangles[(triangles[:, 0] == i) | (triangles[:, 1] == i) | (triangles[:, 2] == i)]

        for j in range(req_t.shape[0]):

            nbhrs = [v for v in req_t[j] if v != i]
            vec1 = vertices[nbhrs[0]] - vertices[i]
            vec1 = vec1 / np.linalg.norm(vec1, 2)
            vec2 = vertices[nbhrs[1]] - vertices[i]
            vec2 = vec2 / np.linalg.norm(vec2, 2)
#             angle = np.arccos(np.dot(vec1, vec2) / (np.linalg.norm(vec1, 2) * np.linalg.norm(vec2, 2)))
            angle = np.arccos(np.dot(vec1, vec2))
            sum_theta+=angle
        K_G[i] = ((2 * np.pi) - sum_theta) / A_mixed[i]
    return K_G

In [9]:
def get_principal_curvatures(K_H, K_G):
    numv = vertices.shape[0]
    numt = triangles.shape[0]
    zeros = np.zeros(numv)
    delx = np.max(np.vstack((K_H**2 - K_G, zeros)), axis=0)
    K_1 = K_H + delx
    K_2 = K_H - delx
    return K_1, K_2

In [10]:
def read_off(file):
    if 'OFF' != file.readline().strip():
        raise('Not a valid OFF header')
    n_verts, n_faces, n_dontknow = tuple([int(s) for s in file.readline().strip().split(' ')])
    verts = [[float(s) for s in file.readline().strip().split(' ')] for i_vert in range(n_verts)]
    faces = [[int(s) for s in file.readline().strip().split(' ')][1:] for i_face in range(n_faces)]
    return np.array(verts, dtype=np.float64), np.array(faces, dtype=np.int64)

In [11]:
f = open("torus.off")
vertices = np.loadtxt("2wcV.txt", dtype=float)
triangles = np.loadtxt("2wcT.txt", dtype=int) - 1
# vertices, triangles = read_off(f)

In [12]:
cot_matrix, scaling_factor = generate_cot_matrix(vertices, triangles)

In [13]:
A_mixed = calc_A_mixed(vertices, cot_matrix, scaling_factor)

In [14]:
K_H = get_mean_curvature(vertices, triangles, A_mixed, cot_matrix)

In [15]:
K_G = get_gaussian_curvature(vertices, triangles, A_mixed)

In [16]:
K_G

array([1.00481178, 1.00477972, 1.00442407, 1.00541911, 1.00572027,
       1.00530024, 1.00213665, 1.00192514, 1.00150831, 1.00542378,
       1.00530391, 1.00564591, 1.00544074, 1.00250839, 1.00209977,
       1.0019751 , 1.00206654, 1.00203837, 1.0048281 , 1.00516734,
       1.00589924, 1.00595288, 1.00446689, 1.00218433, 1.00213378,
       1.00187   , 1.00239761, 1.00244432, 1.00225731, 1.00211773,
       1.00226702, 1.0024646 , 1.00140879, 1.00185245, 1.00521649,
       1.0058665 , 1.00600753, 1.00267086, 1.00221359, 1.00239507,
       1.00209509, 1.00252633, 1.00190719, 1.00209041, 1.00240457,
       1.0031952 , 1.00280107, 1.00286797, 1.00304261, 1.0026128 ,
       1.00220128, 1.00321983, 1.00326082, 1.00199898, 1.00208566,
       1.00253502, 1.00273282, 1.00293803, 1.00287418, 1.00309249,
       1.00309816, 1.0032201 , 1.00273154, 1.00292195, 1.00216932,
       1.00214136, 1.00426044, 1.00391832, 1.00367546, 1.00334123,
       1.00300896, 1.00332243, 1.00391264, 1.00588484, 1.00622

In [21]:
f = np.zeros(triangles.shape[0])
for i in range(f.shape[0]):
    f[i] = np.mean(K_G[triangles[i]])

In [22]:
x, y, z = vertices[:, 0], vertices[:, 1], vertices[:, 2]

mesh = mlab.triangular_mesh(x, y, z, triangles, 
                                    representation='wireframe', opacity=0) 
# mesh.mlab_source.dataset.cell_data.scalars = f 
# mesh.mlab_source.dataset.cell_data.scalars.name = 'Cell data' 
# mesh.mlab_source.update() 
# mesh.parent.update()

# mesh2 = mlab.pipeline.set_active_attribute(mesh, 
#                 cell_scalars='Cell data') 

mesh.mlab_source.dataset.point_data.scalars = f 
mesh.mlab_source.dataset.point_data.scalars.name = 'Cell data' 
mesh.mlab_source.update() 
mesh.parent.update()

mesh2 = mlab.pipeline.set_active_attribute(mesh, 
                point_scalars='Cell data') 
s2 = mlab.pipeline.surface(mesh2) 


mlab.show()

In [61]:
A_mixed

array([0.01958115, 0.01926544, 0.01771312, 0.02204787, 0.02329457,
       0.02157424, 0.00866436, 0.00787393, 0.00560689, 0.02228856,
       0.02168609, 0.02315909, 0.02224175, 0.01038209, 0.00853362,
       0.00758296, 0.00849675, 0.00832932, 0.01947593, 0.02072088,
       0.02387428, 0.02373978, 0.01753953, 0.00875913, 0.00854257,
       0.00760655, 0.00976134, 0.00996483, 0.00895865, 0.00857824,
       0.00925474, 0.00990119, 0.00566534, 0.00717404, 0.02076532,
       0.02362017, 0.02458079, 0.01103669, 0.00898143, 0.00964663,
       0.00833162, 0.0101812 , 0.00741879, 0.00850956, 0.00966972,
       0.01313192, 0.01130788, 0.01148936, 0.01247453, 0.00996416,
       0.00892169, 0.01271433, 0.01308873, 0.00816181, 0.00849932,
       0.01010902, 0.01114863, 0.0119337 , 0.01161682, 0.01224896,
       0.01267987, 0.01293144, 0.01092641, 0.01188373, 0.00853529,
       0.0083305 , 0.01719642, 0.01578314, 0.01507906, 0.0137434 ,
       0.0118887 , 0.01351201, 0.01558807, 0.02403786, 0.02536

In [151]:
cot_matrix.shape

(140, 140)

In [119]:
diff = get_diff(vertices)

In [120]:
cot_matrix * diff

ValueError: operands could not be broadcast together with shapes (6652,6652) (6652,6652,3) 

In [173]:
(np.array([[1, 1], [2, 2], [3, 3]]).reshape(-1, 1) - np.array([[1, 1], [2, 2], [3, 3]]).reshape(1, -1)).reshape(3, 3, 2)

ValueError: cannot reshape array of size 36 into shape (3,3,2)

In [86]:
vertices.shape

(6652, 3)

In [102]:
np.array([[1, 2], [2, 3], [3, 4]]).reshape(1, -1, 2) 

array([[[0, 0],
        [0, 0],
        [0, 0]]])

In [180]:
np.linalg.norm((np.array([[1, 2], [2, 3], [3, 4]]).reshape(-1, 1, 2) - np.array([[1, 2], [2, 3], [3, 4]])), 2, axis=2)**2

array([[0., 2., 8.],
       [2., 0., 2.],
       [8., 2., 0.]])

In [179]:
(np.array([[1, 2], [2, 3], [3, 4]]).reshape(-1, 1, 2) - np.array([[1, 2], [2, 3], [3, 4]]))

array([[[ 0,  0],
        [-1, -1],
        [-2, -2]],

       [[ 1,  1],
        [ 0,  0],
        [-1, -1]],

       [[ 2,  2],
        [ 1,  1],
        [ 0,  0]]])

In [199]:
np.multiply(np.array([[2, 2, 2], [3, 3, 3], [4, 4, 4]]), np.array([[[1, 2, 3], [4, 5, 6], [5, 6, 7]],
            [[1, 2, 3], [4, 5, 6], [5, 6, 7]],
         [[1, 2, 3], [4, 5, 6], [5, 6, 7]]]))

array([[[ 2,  4,  6],
        [12, 15, 18],
        [20, 24, 28]],

       [[ 2,  4,  6],
        [12, 15, 18],
        [20, 24, 28]],

       [[ 2,  4,  6],
        [12, 15, 18],
        [20, 24, 28]]])

In [193]:
np.array([[[1, 2, 3], [4, 5, 6], [5, 6, 7]],
            [[1, 2, 3], [4, 5, 6], [5, 6, 7]],
         [[1, 2, 3], [4, 5, 6], [5, 6, 7]]])

array([[[1, 2, 3],
        [4, 5, 6],
        [5, 6, 7]],

       [[1, 2, 3],
        [4, 5, 6],
        [5, 6, 7]],

       [[1, 2, 3],
        [4, 5, 6],
        [5, 6, 7]]])

In [209]:
a = np.random.randint(0, 10, (10, 3)) 

In [217]:
a

array([[8, 0, 4],
       [0, 1, 1],
       [7, 8, 4],
       [2, 5, 1],
       [9, 9, 2],
       [2, 1, 1],
       [7, 6, 7],
       [3, 5, 2],
       [1, 1, 1],
       [5, 6, 3]])

In [222]:
(np.arange(10) * a.T).T

array([[ 0,  0,  0],
       [ 0,  1,  1],
       [14, 16,  8],
       [ 6, 15,  3],
       [36, 36,  8],
       [10,  5,  5],
       [42, 36, 42],
       [21, 35, 14],
       [ 8,  8,  8],
       [45, 54, 27]])

array([0.97277191, 0.95669373, 0.96090439, 0.94058149, 0.93570323])

In [65]:
i = 0
triangles[(triangles[:, 0] == i) | (triangles[:, 1] == i) | (triangles[:, 2] == i)]

array([[  0, 201, 107],
       [  0, 201, 130],
       [  1,   0, 107],
       [256,   0, 130],
       [  2,   1,   0],
       [  2, 256,   0]])

array([0.01958115, 0.01926544, 0.01771312, 0.02204787, 0.02329457,
       0.02157424, 0.00866436, 0.00787393, 0.00560689, 0.02228856,
       0.02168609, 0.02315909, 0.02224175, 0.01038209, 0.00853362,
       0.00758296, 0.00849675, 0.00832932, 0.01947593, 0.02072088,
       0.02387428, 0.02373978, 0.01753953, 0.00875913, 0.00854257,
       0.00760655, 0.00976134, 0.00996483, 0.00895865, 0.00857824,
       0.00925474, 0.00990119, 0.00566534, 0.00717404, 0.02076532,
       0.02362017, 0.02458079, 0.01103669, 0.00898143, 0.00964663,
       0.00833162, 0.0101812 , 0.00741879, 0.00850956, 0.00966972,
       0.01313192, 0.01130788, 0.01148936, 0.01247453, 0.00996416,
       0.00892169, 0.01271433, 0.01308873, 0.00816181, 0.00849932,
       0.01010902, 0.01114863, 0.0119337 , 0.01161682, 0.01224896,
       0.01267987, 0.01293144, 0.01092641, 0.01188373, 0.00853529,
       0.0083305 , 0.01719642, 0.01578314, 0.01507906, 0.0137434 ,
       0.0118887 , 0.01351201, 0.01558807, 0.02403786, 0.02536